<a href="https://colab.research.google.com/github/JimMiller-0/Might-be-my-Year/blob/main/Might_be_my_Year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook for retreiving Fantasy Football Data from ESPN for analysis and predictive capabilities

Get Data

In [ ]:
# Install necessary SDKs

!pip install espn-api
!pip install google-cloud-secret-manager
!pip install google-cloud-bigquery





Imports

In [34]:
import pandas as pd
from espn_api.football import League
from google.cloud import secretmanager
import requests
import json
import time
import numpy as np
import datetime
import os

League Variables & Access Tokens for ESPN API

In [35]:
# You will need to get your league ID and ESPN S2 and SWID
# See https://github.com/cwendt94/espn-api/wiki/Football-Intro for details
# Recommended: Store SWID and ESPN S2 in a secrets manager, like gcp secrets manager: https://cloud.google.com/security/products/secret-manager

league_id = 1054374 # => set to league ID that you want to pull data from
season=2023 # => set to year you want to pull data from
url=f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?scoringPeriodId=17&view=mBoxscore&view=m' # => url of ESPN API. Note: this has changed over the years, navigate to the site and inspect network calls to get current endpoint

# Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()

#Create a Client for secrets manager
client = secretmanager.SecretManagerServiceClient()
project_id = 'might-be-my-year' # => GCP Project ID where secrets manager is enabled
secret_espn_s2 = 'espn_s2' # name of secret in GCP secrets manager for espn_s2
secret_swid = 'swid' # name of secret in GCP secrets manager for swid

# Forge the paths to the latest version of the secrets with a F-string:
resource_name_espn_s2 = f"projects/{project_id}/secrets/{secret_espn_s2}/versions/latest"
resource_name_swid = f"projects/{project_id}/secrets/{secret_swid}/versions/latest"

# Load up the secrets to a variable at runtime:
response_espn_s2 = client.access_secret_version(name=resource_name_espn_s2)
response_swid = client.access_secret_version(name=resource_name_swid)

espn_s2 = response_espn_s2.payload.data.decode("UTF-8")
swid = response_swid.payload.data.decode("UTF-8")





Get Data From ESPN API

In [ ]:
r = requests.get(url, cookies={'swid': swid, 'espn_s2': espn_s2})
if r.status_code == 200:

  data = r.json()
  data

# Save the data to a JSON file
  with open('data.json', 'w') as f:
        json.dump(data, f, indent=0)  # Use indent for pretty printing
  print('JSON file saved successfully.')
else:
  print('Request failed with status code:', r.status_code)

# Read the JSON file
with open('data.json', 'r') as f:
    data = json.load(f)

data


Get Data in a schema that makes sense because ESPN is crazy.

In [37]:
df = pd.json_normalize(data, record_path=['teams'])
df

,abbrev,divisionId,id,logo,name,rankCalculatedFinal,record.overall.losses,record.overall.ties,record.overall.wins
0,NICE,1,1,https://g.espncdn.com/lm-static/logo-packs/ffl...,A Very Nice Team,11,10,0,4
1,GRTH,1,2,https://i.imgflip.com/6sfrl3.jpg,Blood Type Hot Dog,3,6,0,8
2,DIE,1,3,https://i.redd.it/xru8hir3z73b1.jpg,Pathetic Excuse For A Team,5,5,0,9
3,McD,1,4,https://upload.wikimedia.org/wikipedia/en/thum...,The MacDaddy,9,9,0,5
4,LAR,1,7,https://g.espncdn.com/lm-static/logo-packs/cor...,Return of The King,8,8,0,6
5,#1,1,8,https://g.espncdn.com/lm-static/logo-packs/ffl...,REINING ROGERS,1,7,0,7
6,SP$,1,9,https://g.espncdn.com/lm-static/logo-packs/cor...,Slim Pickens,10,9,0,5
7,BOYZ,1,13,https://g.espncdn.com/lm-static/logo-packs/ffl...,Boys Club,6,7,0,7
8,Poop,1,14,http://entertainment.ie/images_content/rect/ki...,My Team Still Stinks,12,9,0,5
9,ANTI,1,15,https://g.espncdn.com/lm-static/logo-packs/ffl...,Anti-Football Football Club,2,4,0,10


In [52]:
# Get Draft Details

draft_url= f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav'

r = requests.get(draft_url, cookies={'swid': swid, 'espn_s2': espn_s2})
if r.status_code == 200:

  draft_data = r.json()
  draft_data

# Save the data to a JSON file
  with open('draft_data.json', 'w') as f:
        json.dump(draft_data, f, indent=0)  # Use indent for pretty printing
  print('JSON file saved successfully.')
else:
  print('Request failed with status code:', r.status_code)

  # Read the JSON file
with open('draft_data.json', 'r') as f:
    draft_data = json.load(f)

draft_picks_df = pd.json_normalize(draft_data['draftDetail'], record_path=['picks'])
draft_picks_df


JSON file saved successfully.


,autoDraftTypeId,bidAmount,id,keeper,lineupSlotId,nominatingTeamId,overallPickNumber,playerId,reservedForKeeper,roundId,roundPickNumber,teamId,tradeLocked,memberId
0,0,28,1,True,4,0,1,4372016,False,1,1,1,False,NaN
1,0,6,2,True,4,0,2,4426502,False,1,2,1,False,NaN
2,0,14,3,True,4,0,3,4362628,False,1,3,2,False,NaN
3,0,3,4,True,4,0,4,4569618,False,1,4,2,False,NaN
4,0,37,5,True,6,0,5,15847,False,1,5,3,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0,1,176,False,20,1,176,3128390,False,15,8,15,False,{CE729A03-6338-48DE-B29A-036338C8DEBC}
176,0,1,177,False,16,7,177,-16009,False,15,9,4,False,{51569046-C6C8-4BB8-AC70-4F2D1AEDCCCE}
177,0,1,178,False,20,13,178,4692590,False,15,10,7,False,{034F8301-8FBD-47FE-AB27-E7EB9E14609D}
178,0,1,179,False,20,8,179,3929645,False,15,11,7,False,{034F8301-8FBD-47FE-AB27-E7EB9E14609D}


Load Data in BigQuery - This makes is easier to manually slice and dice

In [55]:
from google.cloud import bigquery
import pyarrow as pa
from google.cloud.exceptions import NotFound
import pyarrow.parquet as pq

# set project id again
project_id = 'might-be-my-year'  # Replace with your actual project ID

# Construct a BigQuery client object.
bq_client = bigquery.Client(project=project_id)

# TODO(developer): Set dataset_id to the ID of the dataset to create.
dataset_id = "{}.fantasy_football".format(bq_client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Check if the dataset exists
try:
    bq_client.get_dataset(dataset_id)  # Make an API request.
    print(f"Dataset {dataset_id} already exists.")
except NotFound:
    print(f"Dataset {dataset_id} does not exist. Creating...")
# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
    dataset = bq_client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(bq_client.project, dataset.dataset_id))

# Convert the DataFrame to a Parquet file
table = pa.Table.from_pandas(draft_picks_df)
pq.write_table(table, 'draft_picks.parquet')

# TODO(developer): Set table_id to the ID of the table to create.
draft_picks_table_id = f'{project_id}.fantasy_football.draft_picks_{season}'

job_config = bigquery.LoadJobConfig(
    autodetect=True, source_format=bigquery.SourceFormat.PARQUET
)

# open parquet file -> load the table from parquet file into bq table
with open('draft_picks.parquet', "rb") as source_file:
    job = bq_client.load_table_from_file(
        source_file, draft_picks_table_id, job_config=job_config
    )

# Wait for the job to complete
job.result()

print("Loaded {} rows and {} columns to {}".format(job.output_rows, len(draft_picks_df.columns), draft_picks_table_id))

Dataset might-be-my-year.fantasy_football already exists.
Loaded 180 rows and 14 columns to might-be-my-year.fantasy_football.draft_picks_2023


Analyze Data

Goal: Plot "Winners" and "Losers" based off of draft day auction price vs. production.

should be somthing like avg auction value vs price paid for on draft day vs inferred auction value based on end of year performance

plot 1: avg auction value vs price paid for on draft day. + difference shows league values that player/position more, - difference league values that player/position less

plot 2: avg auction value vs inferred auction value based on end of the year performance. + difference = player outperfomed expectations, - difference = player underperformed expectations

Plot 3: differnece in plot 1 vs difference in plot 2. quadrant plot: q1 =  players who under performed and the league overvalued. q2 = players who out performed and the league over valued. q3 = players who outperformed and the league undervalued. q4 =  players that underperformed and the league undervalued


In [ ]:
#